# Artifact of Table 7 for Phase Moducation Side Channel Analysis and JTAG Attacks

This is part of a series of artifacts. Note these notebooks are designed to run on Google Colab so have been simplified from those available in the [full repository](https://github.com/colinoflynn/phase-modulation-sca).

To run these notebooks, simply run from top to bottom.

**WARNING**: Some steps, such as the extraction, can be very slow. Try not to forget you started the process as CoLab will kill unattended notebooks eventually, and you'll have to start again.

In [1]:
!pip install scared

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.9 MB/s eta 0:00:00


In [2]:
!wget https://www.dropbox.com/scl/fi/dw5flwpo0ky5wqicuunfd/jtag_glitch.7z?rlkey=urhkwqxj21qpnizt4smsjvzqd&dl=1

--2024-07-26 19:14:24--  https://www.dropbox.com/scl/fi/dw5flwpo0ky5wqicuunfd/jtag_glitch.7z?rlkey=urhkwqxj21qpnizt4smsjvzqd
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1ebea834f32ce5194b90bc9ec5.dl.dropboxusercontent.com/cd/0/inline/CXeTWOArHWN9-11EDXASCg0jK1PSp_MRHKkoDDeWO_vnz_9_frGBRuPWcMcT91ato52BykqfZ-B6gZUWVHOHlFo-nylBJP0ApYaRTJVD8ynKtiH_h7IdYDrF-DCGy-BnxYLkgjY3NEmVP1rtRVQ8g5G8/file# [following]
--2024-07-26 19:14:25--  https://uc1ebea834f32ce5194b90bc9ec5.dl.dropboxusercontent.com/cd/0/inline/CXeTWOArHWN9-11EDXASCg0jK1PSp_MRHKkoDDeWO_vnz_9_frGBRuPWcMcT91ato52BykqfZ-B6gZUWVHOHlFo-nylBJP0ApYaRTJVD8ynKtiH_h7IdYDrF-DCGy-BnxYLkgjY3NEmVP1rtRVQ8g5G8/file
Resolving uc1ebea834f32ce5194b90bc9ec5.dl.dropboxusercontent.com (uc1ebea834f32ce5194b90bc9ec5.dl.dropboxusercontent.com)... 162.125.3.15, 2620

In [3]:
!7z x jtag_glitch.7z\?rlkey\=urhkwqxj21qpnizt4smsjvzqd


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 948694294 bytes (905 MiB)

Extracting archive: jtag_glitch.7z?rlkey=urhkwqxj21qpnizt4smsjvzqd
--
Path = jtag_glitch.7z?rlkey=urhkwqxj21qpnizt4smsjvzqd
Type = 7z
Physical Size = 948694294
Headers Size = 629
Method = LZMA2:22
Solid = -
Blocks = 14

  0%      0% 1 - jtag_glitch/jtagglitch_15MHzCPU_158MHz_shuntcpa.ets                                                              1% 1 - jtag_glitch/jtagglitch_15MHzCPU_158MHz_shuntcpa.ets                                                            

In [4]:
!rm -rf  jtag_glitch.7z\?rlkey\=urhkwqxj21qpnizt4smsjvzqd

In [5]:
import scared
import estraces
import numpy as np

In [6]:
knownkey = [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def find_success(att, knownkey):
    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')
    print(recovered_masterkey == knownkey)

    for i in range(0, np.shape(att.convergence_traces)[2]):
        if np.all(np.argmax(att.convergence_traces[:,:,i], axis=0) == knownkey):
            return (i+1)*att.convergence_step
    return -1

In [8]:
import scared
from scipy import signal


def do_cpa(filename, step=25, window=None, hpf=None, lpf=None):
    ths = estraces.read_ths_from_ets_file(filename)
    print(ths)

    selection_function = scared.aes.selection_functions.encrypt.FirstSubBytes()

    att = scared.CPAAttack(selection_function=selection_function,
                           model=scared.HammingWeight(),
                           discriminant=scared.maxabs,
                           convergence_step=step)

    processes = []

    # Simple high pass filter (HPF)
    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')

        @scared.preprocess
        def hpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(hpfproc)

    # Simple low pass filter (LPF)
    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')

        @scared.preprocess
        def lpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(lpfproc)

    # This resync preprocessing isn't really used, was used with some JTAG captures only
    @scared.preprocess
    def resync(traces):
        import numpy as np
        first = []
        for w in traces:
            first.append(w[0])
        avg = np.mean(first)

        tracesresync = []
        for w in traces:
            if w[0] < avg:
                tracesresync.append(w[:-1])
            else:
                tracesresync.append(w[1:])
        return np.array(tracesresync)
    #processes.append(resync)

    if window:
        container = scared.Container(ths, frame=range(window[0], window[1]), preprocesses=processes)
    else:
        container = scared.Container(ths, preprocesses=processes)


    att.run(container)

    import numpy as np

    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')

    return att

In [12]:
# Datasets in baseline_traces_cpa online
filenames = [
              ('jtag_glitch/jtagglitch_15MHzCPU_159MHz_async_10koffsetLA_mixerjtagboard_jtaglaresults_k.ets', 500, (3600,9000), None, None),
              ('jtag_glitch/jtagglitch_15MHzCPU_159MHz_10koffsetLA_mixerjtagboard_jtaglaresults_60k.ets', 500, (6000,12500), None, None),
            ]

## Doing all the CPA attacks on JTAG Fault Sensitivity measurements

The following will do all the attacks on *fault sensitivity* measurements. The results get saved to a `.npy` file for further use. The `.npy` files have also been saved from prior runs.

In [13]:
for f in filenames:
    att = do_cpa(f[0], f[1], f[2], f[3], f[4])
    successnum = find_success(att, knownkey)
    print("{:s} = {:d}".format(f[0], successnum))
    np.save(f[0] + ".npy", att)

Trace Header Set:
Name.............: ETS Format THS
Reader...........: ETS format reader of file jtag_glitch/jtagglitch_15MHzCPU_159MHz_async_10koffsetLA_mixerjtagboard_jtaglaresults_k.ets with 180387 traces.
ciphertext.......: uint8
key..............: uint8
plaintext........: uint8

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
jtag_glitch/jtagglitch_15MHzCPU_159MHz_async_10koffsetLA_mixerjtagboard_jtaglaresults_k.ets = 86500
Trace Header Set:
Name.............: ETS Format THS
Reader...........: ETS format reader of file jtag_glitch/jtagglitch_15MHzCPU_159MHz_10koffsetLA_mixerjtagboard_jtaglaresults_60k.ets with 73785 traces.
ciphertext.......: uint8
key..............: uint8
plaintext........: uint8

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
jtag_glitch/jtagglitch_15MHzCPU_159MHz_10koffsetLA_mixerjtagboard_jtaglaresults_60k.ets = 41500


In [ ]:
%%bash
mkdir results
mv baseline_traces_cpa/*.npy results/.

In [ ]:
!7z a results.7z results

The above results in a bunch of output, but the important parts are the lines that look like this:

```
jtag_glitch/jtagglitch_15MHzCPU_159MHz_async_10koffsetLA_mixerjtagboard_jtaglaresults_k.ets = 86500
jtag_glitch/jtagglitch_15MHzCPU_159MHz_10koffsetLA_mixerjtagboard_jtaglaresults_60k.ets = 41500
```

The above can be found in Table 7, showing the CPA results of the JTAG fault sensitivity analysis. We'll recreate the T-Test results below.

You may wish to save the .npy files. While we have a copy of them, for a "complete" artifact recreation you should copy the .npy files to the CPA plotting notebook.

The easiest way to do this if using colab (and not a local Jupyter install) is to copy them to a directory & compress them. Then download the file in the CoLab web interface.

The following block will move them to a `results` directory, then the block after that will compress that directory. Look for a `results.7z` file you download.

In [ ]:
%%bash
mkdir results
mv baseline_traces_cpa/*.npy results/.

In [ ]:
!7z a results.7z results

## T-Test Results

The following recreats the T-Test results for Table 7.

In [14]:
import os
import numpy as np
def plot_t(t_val, N, titledata=""):
    import matplotlib.pylab as plt
    ldata = len(t_val)
    plt.plot(t_val)
    plt.plot([0, ldata], [4.5, 4.5], 'k--')
    plt.plot([0, ldata], [-4.5, -4.5], 'k--')
    plt.title("TVLA Results, N=%d, %s"%(N, titledata))
    plt.xlabel("Sample Number")
    plt.ylabel("T-Test Value")

#Set root data dir, not "mixers"
data_dir = "."

def npload(filename):
    return np.load(os.path.join(data_dir, filename), allow_pickle=True)

def nptload(prefix):
    group1 = npload(prefix + "_tvla_group1.npy")
    group2 = npload(prefix + "_tvla_group2.npy")
    N = round((len(group1) + len(group2)) / 2)
    return group1, group2, N

def npsave(filename, array):
    np.save(os.path.join(data_dir, filename), array)

def nptsave(prefix, group1, group2):
    npsave(prefix + "_tvla_group1.npy", group1)
    npsave(prefix + "_tvla_group2.npy", group2)


In [15]:
files = [
         [r"jtag_glitch/jtag_offset10000_clock159E6_10k_xor_extclock_new", 0, None, None, None],
         [r"jtag_glitch/jtag_offset10000_clock159E6_10k_xor_async", 0, None, None, None],
        ]

In [16]:
from scipy.stats import ttest_ind
import matplotlib.pylab as plt
from scipy import signal

def find_max_t(file, start=0, end=-1, hpf=None, lpf=None, resync_jtag=False):
    group1, group2, N = nptload(file)

    if resync_jtag:
        first = []
        for g in group1:
            first.append(g[0])
        avg = np.mean(first)
        group1resync = []
        for g in group1:
            if g[0] > avg:
                group1resync.append(g[:-1])
            else:
                group1resync.append(g[1:])

        group2resync = []
        for g in group2:
            if g[0] > avg:
                group2resync.append(g[:-1])
            else:
                group2resync.append(g[1:])

        group1 = group1resync
        group2 = group2resync

    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    t_val = ttest_ind(group1[start:end], group2[start:end], axis=0, equal_var=False)[0]
    max_t = max(abs(t_val))
    return max_t, t_val
for f in files:
    max_t, t_val = find_max_t(f[0], start=f[1], end=f[2], hpf=f[3], lpf=f[4])
    print("{:s}: {:f}".format(f[0], max_t))

jtag_glitch/jtag_offset10000_clock159E6_10k_xor_extclock_new: 4.669240
jtag_glitch/jtag_offset10000_clock159E6_10k_xor_async: 5.015154


This outputs the T-Test results. You can confirm the values match the table above (Table 7 in paper):

```
jtag_glitch/jtag_offset10000_clock159E6_10k_xor_extclock_new: 4.669240
jtag_glitch/jtag_offset10000_clock159E6_10k_xor_async: 5.015154
```

Where "jtag_offset10000_clock159E6_10k_xor_extclock_new" is "sync", and "jtag_offset10000_clock159E6_10k_xor_async" is "Async.

This should now be Table 7 fully reproduced, both CPA and T-Test values.